## Что первое пришло в голову ##

Блин, не применить tf/idf, ngram и word2vec (хотя про последнее не факт, но я ещё не придумал как)

Необычная NLP задача, по факту нужно классифицировать класс каждой буквы в слове на основании каких-то признаков.


Потенциальные признаки: векторное представление слова (циферка для каждой буквы, желательно по-умному нормализовать, потому что нормализовать по алфавиту - не очень, скорее всего нужно будет присваивать цифру в зависимости от частоты встречаемости), длина слова, число гласных/согласных, позиция слова в изначальном названии компании, внутри ли кавычек слово.

Дальше можно будет проверить разные классификаторы.

In [1]:
import numpy as np
import pandas as pd

# Длина максимального слова (для нормализации), подробнее в 'Testing.ipynb'
MAX_WORD = 57

CONSONANTS = 'бвгджзйклмнпрстфхцчшщБВГДЖЗЙКЛМНПРСТФХЦЧШЩbcdfghjklmnpqrstvwxyzBCDFGHJKLMNPQRSTVWXZ'
VOWELS = 'аиоуэыеёюяАИОУЭЫЕЁЮЯaeiouyAEIOUY'
ALPHABET = 'АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯабвгдеёжзийклмнопрстуфхцчшщъыьэюяaeiouyAEIOUYbcdfghjklmnpqrstvwxyzBCDFGHJKLMNPQRSTVWXZ'

In [1]:
train_companies = open('../kontur_srs_internship_test_task/train.txt', 'r').readlines()
train_companies = list(map(lambda s: s.strip(), train_companies))

test_companies = open('../kontur_srs_internship_test_task/test.txt', 'r').readlines()
test_companies = list(map(lambda s: s.strip(), test_companies))

In [5]:
with open('input.txt', 'w') as result:
    for company in train_companies[:100000] :
        result.write(company +'\n')

In [8]:
len(test_companies)

581387

In [7]:
with open('test.lower.txt', 'w') as result:
    for company in test_companies[:10000]:
        result.write(company.lower() +'\n')

In [3]:
import re
               
def tokenize(company, spaces=False):
    with_spaces = list(filter(None, re.split('(\w+| )', company)))
    if(spaces):
        return with_spaces
    return [token for token in with_spaces if token != ' ']

def has_letters(string):
    return any(char in ALPHABET for char in string)

def has_not_letters(string):
    return any(char not in ALPHABET for char in string)

def count_vowels(string):
    return len([char for char in string if char in VOWELS])

def count_consonants(string):
    return len([char for char in string if char in CONSONANTS])

В данной строке артефакт, подробнее в 'Analysis.ipynb'

In [4]:
del train_companies[471970]

In [5]:
def extract_features(company, test=False):
    
    # Длина слова, количество гласных/согласных, положение в слове,
    # есть ли в слове что-то помимо букв и то же самое для предыдущего
    # и последующего слова
    num_of_features = 21
    
    # 3 класса - полностью писменными, первая заглавная или полностью заглавными
    num_of_classes = 3

    size = 0
    for word in tokenize(company):
        if(has_letters(word)):
            size += 1
            
    features = np.zeros((size, num_of_features))
    labels = np.zeros(size)
    
    index = 0
    tokens = tokenize(company)
    for i, word in enumerate(tokens):
        if(has_letters(word)):
            
            if(index == 0):
                features[index][7] = index/size
                features[index][8] = len(word)/MAX_WORD
                features[index][9] = count_consonants(word)/len(word)
                features[index][10] = count_vowels(word)/len(word)
                features[index][11] = int(has_not_letters(word))
                if(i > 0):
                    features[index][12] = int(tokens[i - 1] == '"')
                    features[index][13] = int(tokens[i - 1] == '(')
                
            else:
                features[index - 1][14] = index/size
                features[index - 1][15] = len(word)/MAX_WORD
                features[index - 1][16] = count_consonants(word)/len(word)
                features[index - 1][17] = count_vowels(word)/len(word)
                features[index - 1][18] = int(has_not_letters(word))
                features[index - 1][19] = int(tokens[i - 1] == '"')
                features[index - 1][20] = int(tokens[i - 1] == '(')
                
                features[index][:14] = features[index - 1][7:]
            
            if(word[0].isupper()):
                if(word.isupper()):
                    labels[index] = 2
                else:
                    labels[index] = 1
            else:
                labels[index] = 0
            index += 1
    if(not test):    
        return list(features), list(labels)
    else:
        return list(features)

In [17]:
def extract_features(company, test=False):
    
    # Длина слова, количество гласных/согласных, положение в слове,
    # есть ли в слове что-то помимо букв и то же самое для предыдущего
    # и последующего слова
    num_of_features = 11 + MAX_WORD
    length = num_of_features * 3
    
    # 3 класса - полностью писменными, первая заглавная или полностью заглавными
    num_of_classes = 3

    size = 0
    for word in tokenize(company):
        if(has_letters(word)):
            size += 1
            
    features = np.zeros((size, length))
    labels = np.zeros(size)
    
    index = 0
    tokens = tokenize(company, spaces=True)
    for i, word in enumerate(tokens):
        if(has_letters(word)):
            if(index == 0):
                features[index][num_of_features] = index/size
                features[index][num_of_features + 1] = len(word)/MAX_WORD
                features[index][num_of_features + 2] = count_consonants(word)/len(word)
                features[index][num_of_features + 3] = count_vowels(word)/len(word)
                features[index][num_of_features + 4] = int(has_not_letters(word))
                if(i > 0):
                    features[index][num_of_features + 5] = int(tokens[i - 1] == '"')
                    features[index][num_of_features + 6] = int(tokens[i - 1] == '(')
                    features[index][num_of_features + 7] = int(tokens[i - 1] == ' ')
                if( i + 1 < len(tokens)):
                    features[index][num_of_features + 8] = int(tokens[i + 1] == ')')
                    features[index][num_of_features + 9] = int(tokens[i + 1] == '"')
                    features[index][num_of_features + 10] = int(tokens[i + 1] == ' ')
                    
                for j, char in enumerate(list(word)):
                    features[index][num_of_features + 11 + j] = encoding[char.upper()]
                
            else:
                features[index - 1][2 * num_of_features] = index/size
                features[index - 1][2 * num_of_features + 1] = len(word)/MAX_WORD
                features[index - 1][2 * num_of_features + 2] = count_consonants(word)/len(word)
                features[index - 1][2 * num_of_features + 3] = count_vowels(word)/len(word)
                features[index - 1][2 * num_of_features + 4] = int(has_not_letters(word))
                features[index - 1][2 * num_of_features + 5] = int(tokens[i - 1] == '"')
                features[index - 1][2 * num_of_features + 6] = int(tokens[i - 1] == '(')
                features[index - 1][2 * num_of_features + 7] = int(tokens[i - 1] == ' ')
                if( i + 1 < len(tokens)):
                    features[index - 1][2 * num_of_features + 8] = int(tokens[i + 1] == ')')
                    features[index - 1][2 * num_of_features + 9] = int(tokens[i + 1] == '"')
                    features[index - 1][2 * num_of_features + 10] = int(tokens[i + 1] == ' ')
                
                for j, char in enumerate(list(word)):
                    features[index - 1][2 * num_of_features + 11 + j] = encoding[char.upper()]
                
                features[index][:2 * num_of_features] = features[index - 1][num_of_features:]
            
            if(word[0].isupper()):
                if(word.isupper()):
                    labels[index] = 2
                else:
                    labels[index] = 1
            else:
                labels[index] = 0
            index += 1
    if(not test):    
        return list(features), list(labels)
    else:
        return list(features)

In [8]:
from collections import Counter

# Все подсчитаны все символы в верхнем регистре в трейн сете (из Analysis.ipynb)

counter = Counter({'О': 35312904, ' ': 25422815, 'Н': 24537989, 'Е': 23969940, 'Т': 21420192, 'С': 19892386, 'А': 16809787, 'И': 14950389, 'Р': 13874547, 'В': 13280775, '"': 9695019, 'К': 7858626, 'Г': 5914467, 'Л': 5851872, 'Б': 5428314, 'Й': 5352151, '\n': 5232476, 'Ь': 4866345, 'П': 4642709, 'Д': 4606840, 'М': 4369257, 'Ч': 3898716, 'Я': 3613597, 'У': 3592997, 'Щ': 3302510, 'Ю': 3018424, 'З': 2625035, 'Ц': 2040518, 'Ы': 1500303, '-': 1443125, 'Ж': 1273203, 'Ф': 1136416, 'Х': 1122008, 'Ш': 674331, 'Э': 502713, '.': 406987, '№': 214440, '1': 194707, '2': 167236, '(': 124233, ')': 124117, '0': 102588, ',': 95619, '3': 92674, '4': 73813, '5': 68067, '7': 58772, '9': 58326, '6': 56049, 'Ъ': 55448, '8': 47595, 'Ё': 29626, 'I': 29581, '/': 28983, '+': 25175, 'X': 13675, "'": 12332, '`': 8071, 'N': 7230, 'T': 5980, 'V': 5571, 'O': 5074, 'A': 5020, 'E': 4842, 'R': 4648, 'L': 4577, 'S': 4524, 'C': 4156, 'P': 3837, 'D': 3460, 'M': 2589, ':': 1590, '!': 1578, '&': 1530, 'U': 1491, 'G': 1320, '<': 1152, 'H': 1102, '>': 1069, 'F': 1058, 'Y': 1054, 'B': 1005, 'K': 927, ';': 892, '\\': 634, 'W': 541, '°': 509, '«': 427, 'Z': 379, '»': 378, '*': 374, '%': 237, '_': 237, 'J': 226, '?': 120, 'Q': 58, '=': 48, '@': 45, '–': 41, '^': 29, '{': 24, '}': 23, '[': 16, '~': 16, ']': 14, '$': 12, '#': 12, '‹': 10, '›': 10, '·': 8, '|': 7, '“': 6, '”': 6, '©': 3, '—': 2, '§': 2, '’': 2, '™': 2, '®': 1, 'Є': 1, '\xa0': 1, '±': 1, '…': 1, '¶': 1})
print(len(counter))
encoding = {}
i = 0
for char in counter:
    encoding[char] = i/len(counter)
    i += 1

124


In [9]:
def generate_dataset(companies, test=False):
    features = np.array([])
    labels = np.array([])
    for company in companies:
        curr_features, curr_labels = extract_features(company)
        if(len(features)):
            features += curr_features
            labels += curr_labels
        else:
            features = curr_features
            labels = curr_labels
    if(not test):    
        return features, labels
    else:
        return features 

In [10]:
def transform(company, labels):
    word_index = 0
    corrected = tokenize(company, spaces=True)
    for i, word in enumerate(corrected):
        if (has_letters(word)):
            if (labels[word_index] == 0):
                corrected[i] = word.lower()
            elif(labels[word_index] == 1):
                corrected[i] = word[0] + word[1:].lower()
            word_index += 1
    return ''.join(corrected)

In [11]:
from sklearn.metrics import f1_score

def score(true_name, pred_name):
    length = len(true_name) 
    true_register = np.zeros(length)
    pred_register = np.zeros(length)
    for i in range(length):
        true_register[i] = int(true_name[i].isupper())
        pred_register[i] = int(pred_name[i].isupper())
    return f1_score(true_register, pred_register)

In [18]:
from sklearn.model_selection import train_test_split
X, y = generate_dataset(train_companies[:100000])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

МППБиО
райСББЖ
райСББЖ
райСББЖ
АссКон
ШЭФиИТ
АдыгеяГАЗавтосервис
КубГТУ
АвиаГСМсервис
АдыгМрамор
ЮгЦентральПроД
ЮгЦентральПроД
МедЭстетПластика
АгроХимСервис
АгроНова
ВторМет
МагаДент
ТехАвто
АльбаЗихия
КаСаДа
АдыгеяМаркет
ИнфоТел
ФинАудит
МЦРТДиЮ
хозяйстваТахтамукайского
АвтоГаз
УниКом
СельхозИнвест
ЯБлоновский
МЦДЮТур
ЦПМиСП
ПромТрансБанк
УралКапиталБанк
БАшкортостана
СТЕКЛОНиТ
БРВПиС
БелАвтодеталь
ЗапУралТИСИЗ
ЗапУралТИСИЗ
БирГСПА
ЖилСтрой
БашЦика
АгроБашСакс
БашкортостанБ
МиЛеДи
БашОптСпецСервис
БашОйл
БашОйл
АВТОРусь
АВТОРусь
БашОйлСервис
ПСРЦдН
АДИПиИ
ВысТех
АгроСпецСервис
ЭкоСтиль
РостАгро
БашАгроПродСервис
ТехГазСервис
КРКДПиМ
ГУПКалтасинское
ТЦСПСиД
предприятияСГПП
ответственостьюПМК
ПУЖКХСалаватского
ЖилСтройСервис
РемТехСервис
СервисАгро
АгриКо
АкБАРС
АкБАРС
ПромСтройКамень
СтройДизайн
БашПромСтрой
ТитанМедТех
ТитанМедТех
СпецСтройТехнология
ПродЭко
ЭрЭмСи
ЭрЭмСи
АРТель
ЭкоСфера
БашкирХИМИКАТ
БашкирХИМИКАТ
ПромТехЦентр
БашПродКоп
ИнтерПромТехнология
БашПромТехнологии
СтройТр

РегионЖилСтрой
ПромТехЗащита
НефтеГазСтройКомплект
ЭкологБашспецнефтестрой
УралТрейдСервис
УралИнтерКом
УфаСпецМонтаж
УфаСтройПроект
ЭммиКАРД
ЭммиКАРД
ЗапУралТИСИЗ
АвиаПром
СтройЭнергоРемонт
ПродОпт
ЮрСервис
ЮрСервис
СпецТэк
УралКарт
МобилКонсалт
СотаТрейд
СотаТрейд
БашУралВест
СПИДом
СПИДом
СПИДом
СПИДом
АвтоматикаСтройСервис
ФинВест
УралТрейдОйл
БиоМедЦентр
БашКЭБ
УфаОйл
КислоХром
УралСибИзыскания
УралТрубоСталь
ЭнергоГазМонтажПусконаладка
СтройГарантСервис
БашВолготанкер
БашВолготанкер
БашЭнергоМонтаж
ТрансАгроКомТех
ЭКСПОРесурс
ЭКСПОРесурс
СтройМеталлЭлектроКомплект
УралСервисПромКомплект
ТелеКом
ТелеКом
ТехноИндустрия
ДомСервис
БашДез
СовГавань
ЭкологияТехно
БашНАФТА
ТелеКомТур
ТелеКомТур
БашОгнеупор
БашОгнеупор
УралСибНефтепроект
УралПромТранс
УралАвтоматикаСервис
ТелеРосс
ТелеРосс
БашИнноватСтрой
ДиамантСтройСервис
ДиамантСтройСервис
МонРемСтрой
ПромСтройРезерв
ЕвроАвтоЗапчасть
ГеоКомплекс
УралСтройКомп
БашИнвестГрупп
СтройЛиния
ГлавСтройКомплект
ЦБМТиВС
БашПолимер
БашСпецМонтаж

ГосстройАПЗпроект
ГосстройАПЗпроект
ГородЪ
АвтоРайт
АвтоПассаж
НовоТекс
ЭнЭрДжи
КарНИИЛП
АвтоЗИП
КарлисТрансСервис
ТермоСтрой
МТСервис
ПетроГИЦ
ТрансФид


In [12]:
from catboost import CatBoostClassifier
from catboost import Pool, cv

# cv_data = generate_dataset(train_companies[:200000])

# cv_dataset = Pool(data=X_train,
#                   label=y_train)

# params = {"iterations": list(range(100, 1001, 50)),
#           "depth": list(range(4,11)),
#           "loss_function": "Logloss",
#           "verbose": False}

# scores = cv(cv_dataset,
#             params,
#             fold_count=5, 
#             plot="True")

clf = CatBoostClassifier(iterations=1000, od_type='IncToDec', task_type='GPU')
clf.fit(X_train, y_train)

Learning rate set to 0.236769
0:	learn: 0.8405496	total: 33.6ms	remaining: 33.6s
1:	learn: 0.6938444	total: 63ms	remaining: 31.5s
2:	learn: 0.6009222	total: 92.8ms	remaining: 30.8s
3:	learn: 0.5321785	total: 119ms	remaining: 29.6s
4:	learn: 0.4811967	total: 149ms	remaining: 29.6s
5:	learn: 0.4434703	total: 178ms	remaining: 29.5s
6:	learn: 0.4129890	total: 204ms	remaining: 28.9s
7:	learn: 0.3889352	total: 231ms	remaining: 28.6s
8:	learn: 0.3705045	total: 261ms	remaining: 28.8s
9:	learn: 0.3537642	total: 291ms	remaining: 28.8s
10:	learn: 0.3398336	total: 316ms	remaining: 28.4s
11:	learn: 0.3269484	total: 344ms	remaining: 28.3s
12:	learn: 0.3181358	total: 366ms	remaining: 27.8s
13:	learn: 0.3081352	total: 392ms	remaining: 27.6s
14:	learn: 0.3013990	total: 413ms	remaining: 27.1s
15:	learn: 0.2938799	total: 442ms	remaining: 27.2s
16:	learn: 0.2887522	total: 471ms	remaining: 27.2s
17:	learn: 0.2815039	total: 501ms	remaining: 27.3s
18:	learn: 0.2779639	total: 526ms	remaining: 27.2s
19:	learn:

166:	learn: 0.1497491	total: 4.28s	remaining: 21.3s
167:	learn: 0.1496728	total: 4.3s	remaining: 21.3s
168:	learn: 0.1495921	total: 4.32s	remaining: 21.2s
169:	learn: 0.1494644	total: 4.34s	remaining: 21.2s
170:	learn: 0.1491577	total: 4.36s	remaining: 21.2s
171:	learn: 0.1490516	total: 4.39s	remaining: 21.1s
172:	learn: 0.1488077	total: 4.41s	remaining: 21.1s
173:	learn: 0.1486710	total: 4.44s	remaining: 21.1s
174:	learn: 0.1484687	total: 4.47s	remaining: 21.1s
175:	learn: 0.1483618	total: 4.5s	remaining: 21.1s
176:	learn: 0.1483037	total: 4.52s	remaining: 21s
177:	learn: 0.1481632	total: 4.55s	remaining: 21s
178:	learn: 0.1479737	total: 4.57s	remaining: 21s
179:	learn: 0.1478881	total: 4.6s	remaining: 20.9s
180:	learn: 0.1476873	total: 4.63s	remaining: 20.9s
181:	learn: 0.1475044	total: 4.66s	remaining: 20.9s
182:	learn: 0.1473434	total: 4.68s	remaining: 20.9s
183:	learn: 0.1471595	total: 4.71s	remaining: 20.9s
184:	learn: 0.1469534	total: 4.74s	remaining: 20.9s
185:	learn: 0.1468040

329:	learn: 0.1343777	total: 8.09s	remaining: 16.4s
330:	learn: 0.1343485	total: 8.11s	remaining: 16.4s
331:	learn: 0.1343004	total: 8.13s	remaining: 16.4s
332:	learn: 0.1342145	total: 8.16s	remaining: 16.3s
333:	learn: 0.1341140	total: 8.19s	remaining: 16.3s
334:	learn: 0.1340817	total: 8.22s	remaining: 16.3s
335:	learn: 0.1340279	total: 8.24s	remaining: 16.3s
336:	learn: 0.1339820	total: 8.27s	remaining: 16.3s
337:	learn: 0.1339604	total: 8.29s	remaining: 16.2s
338:	learn: 0.1339183	total: 8.31s	remaining: 16.2s
339:	learn: 0.1339126	total: 8.33s	remaining: 16.2s
340:	learn: 0.1338811	total: 8.35s	remaining: 16.1s
341:	learn: 0.1337477	total: 8.37s	remaining: 16.1s
342:	learn: 0.1337276	total: 8.39s	remaining: 16.1s
343:	learn: 0.1336829	total: 8.42s	remaining: 16.1s
344:	learn: 0.1336054	total: 8.44s	remaining: 16s
345:	learn: 0.1335643	total: 8.46s	remaining: 16s
346:	learn: 0.1335357	total: 8.48s	remaining: 16s
347:	learn: 0.1334310	total: 8.52s	remaining: 16s
348:	learn: 0.133407

496:	learn: 0.1274743	total: 11.9s	remaining: 12.1s
497:	learn: 0.1274038	total: 12s	remaining: 12.1s
498:	learn: 0.1273965	total: 12s	remaining: 12s
499:	learn: 0.1273918	total: 12s	remaining: 12s
500:	learn: 0.1273010	total: 12.1s	remaining: 12s
501:	learn: 0.1272564	total: 12.1s	remaining: 12s
502:	learn: 0.1272226	total: 12.1s	remaining: 12s
503:	learn: 0.1271929	total: 12.1s	remaining: 11.9s
504:	learn: 0.1271194	total: 12.2s	remaining: 11.9s
505:	learn: 0.1270963	total: 12.2s	remaining: 11.9s
506:	learn: 0.1270842	total: 12.2s	remaining: 11.9s
507:	learn: 0.1270730	total: 12.2s	remaining: 11.8s
508:	learn: 0.1270433	total: 12.2s	remaining: 11.8s
509:	learn: 0.1270189	total: 12.3s	remaining: 11.8s
510:	learn: 0.1270065	total: 12.3s	remaining: 11.8s
511:	learn: 0.1269938	total: 12.3s	remaining: 11.7s
512:	learn: 0.1269873	total: 12.3s	remaining: 11.7s
513:	learn: 0.1269714	total: 12.4s	remaining: 11.7s
514:	learn: 0.1269577	total: 12.4s	remaining: 11.7s
515:	learn: 0.1269478	total:

658:	learn: 0.1225288	total: 15.8s	remaining: 8.18s
659:	learn: 0.1225016	total: 15.8s	remaining: 8.16s
660:	learn: 0.1224915	total: 15.9s	remaining: 8.13s
661:	learn: 0.1224790	total: 15.9s	remaining: 8.11s
662:	learn: 0.1224586	total: 15.9s	remaining: 8.08s
663:	learn: 0.1224277	total: 15.9s	remaining: 8.05s
664:	learn: 0.1223986	total: 15.9s	remaining: 8.03s
665:	learn: 0.1223820	total: 16s	remaining: 8.01s
666:	learn: 0.1223584	total: 16s	remaining: 7.98s
667:	learn: 0.1223431	total: 16s	remaining: 7.96s
668:	learn: 0.1223348	total: 16s	remaining: 7.93s
669:	learn: 0.1223086	total: 16.1s	remaining: 7.91s
670:	learn: 0.1222673	total: 16.1s	remaining: 7.89s
671:	learn: 0.1222410	total: 16.1s	remaining: 7.87s
672:	learn: 0.1222099	total: 16.1s	remaining: 7.84s
673:	learn: 0.1221669	total: 16.2s	remaining: 7.81s
674:	learn: 0.1221599	total: 16.2s	remaining: 7.79s
675:	learn: 0.1220523	total: 16.2s	remaining: 7.76s
676:	learn: 0.1220122	total: 16.2s	remaining: 7.74s
677:	learn: 0.121998

820:	learn: 0.1186859	total: 19.4s	remaining: 4.22s
821:	learn: 0.1186303	total: 19.4s	remaining: 4.2s
822:	learn: 0.1186227	total: 19.4s	remaining: 4.18s
823:	learn: 0.1185977	total: 19.4s	remaining: 4.15s
824:	learn: 0.1185629	total: 19.5s	remaining: 4.13s
825:	learn: 0.1185307	total: 19.5s	remaining: 4.1s
826:	learn: 0.1184698	total: 19.5s	remaining: 4.08s
827:	learn: 0.1184278	total: 19.5s	remaining: 4.06s
828:	learn: 0.1184172	total: 19.6s	remaining: 4.03s
829:	learn: 0.1184051	total: 19.6s	remaining: 4.01s
830:	learn: 0.1183718	total: 19.6s	remaining: 3.99s
831:	learn: 0.1183614	total: 19.6s	remaining: 3.96s
832:	learn: 0.1183500	total: 19.6s	remaining: 3.94s
833:	learn: 0.1182647	total: 19.7s	remaining: 3.92s
834:	learn: 0.1182548	total: 19.7s	remaining: 3.89s
835:	learn: 0.1182380	total: 19.7s	remaining: 3.87s
836:	learn: 0.1182284	total: 19.7s	remaining: 3.84s
837:	learn: 0.1181940	total: 19.8s	remaining: 3.82s
838:	learn: 0.1181591	total: 19.8s	remaining: 3.8s
839:	learn: 0.1

980:	learn: 0.1151562	total: 22.8s	remaining: 441ms
981:	learn: 0.1151292	total: 22.8s	remaining: 418ms
982:	learn: 0.1151039	total: 22.8s	remaining: 395ms
983:	learn: 0.1150798	total: 22.8s	remaining: 372ms
984:	learn: 0.1150576	total: 22.9s	remaining: 348ms
985:	learn: 0.1150438	total: 22.9s	remaining: 325ms
986:	learn: 0.1150151	total: 22.9s	remaining: 302ms
987:	learn: 0.1150077	total: 22.9s	remaining: 278ms
988:	learn: 0.1150008	total: 22.9s	remaining: 255ms
989:	learn: 0.1149933	total: 23s	remaining: 232ms
990:	learn: 0.1149496	total: 23s	remaining: 209ms
991:	learn: 0.1148919	total: 23s	remaining: 186ms
992:	learn: 0.1148742	total: 23s	remaining: 162ms
993:	learn: 0.1148679	total: 23s	remaining: 139ms
994:	learn: 0.1148326	total: 23.1s	remaining: 116ms
995:	learn: 0.1148201	total: 23.1s	remaining: 92.7ms
996:	learn: 0.1148099	total: 23.1s	remaining: 69.5ms
997:	learn: 0.1147941	total: 23.1s	remaining: 46.3ms
998:	learn: 0.1147809	total: 23.1s	remaining: 23.2ms
999:	learn: 0.1147

In [28]:
company = 'ОАО "МЕСЯГУТОВСКИЙ КСОМ"'
extract_features(company)[1]

[2.0, 2.0, 2.0]

In [19]:
for char in 'ООО':
    print(char)

О
О
О


In [50]:
transform(company, clf.predict(extract_features(company, test=True)))

'ГУ Центр госсанэпиднадзора в Кукморском районе РТ'

In [35]:
#1500
true_name = ' '.join(train_companies[2000000:2100000])
predictions = []
for company in train_companies[2000000:2100000]:
     predictions.append(transform(company.upper(), clf.predict(extract_features(company, test=True))))
        
pred_name = ' '.join(predictions)
print(score(true_name, pred_name))

0.7069309101696772


In [13]:
#1000
true_name = ' '.join(train_companies[2000000:2100000])
predictions = []
for company in train_companies[2000000:2100000]:
     predictions.append(transform(company.upper(), clf.predict(extract_features(company, test=True))))
        
pred_name = ' '.join(predictions)
print(score(true_name, pred_name))

0.7227097411312081


In [54]:
#1000 without ( and ""
true_name = ' '.join(train_companies[2000000:2100000])
predictions = []
for company in train_companies[2000000:2100000]:
     predictions.append(transform(company.upper(), clf.predict(extract_features(company, test=True))))
        
pred_name = ' '.join(predictions)
print(score(true_name, pred_name))

0.7074405845966393


In [39]:
#500
true_name = ' '.join(train_companies[2000000:2100000])
predictions = []
for company in train_companies[2000000:2100000]:
     predictions.append(transform(company.upper(), clf.predict(extract_features(company, test=True))))
        
pred_name = ' '.join(predictions)
print(score(true_name, pred_name))

0.7063494110566589


In [14]:
for company in test_companies[:5]:
    print(company)
    print(transform(company, clf.predict(extract_features(company, test=True))))

ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ РАДИОКОМПАНИЯ "ВОСТОК"
Общество с ограниченной ответственностью Радиокомпания "Восток"
ГБУ "АЗНАКАЙЛЕС"
ГБУ "Азнакайлес"
ОАО "АЗНАКАЕВСКОЕ ПАТП"
ОАО "Азнакаевское патп"
ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "АЗНАКАЕВСКОЕ ПАССАЖИРСКОЕ АВТОТРАНСПОРТНОЕ ПРЕДПРИЯТИЕ-1"
Общество с ограниченной ответственностью "Азнакаевское пассажирское автотранспортное предприятие-1"
ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "АЛНАЗ"
Общество с ограниченной ответственностью "Алназ"


In [53]:
test_companies[:5]

['ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ РАДИОКОМПАНИЯ "ВОСТОК"',
 'ГБУ "АЗНАКАЙЛЕС"',
 'ОАО "АЗНАКАЕВСКОЕ ПАТП"',
 'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "АЗНАКАЕВСКОЕ ПАССАЖИРСКОЕ АВТОТРАНСПОРТНОЕ ПРЕДПРИЯТИЕ-1"',
 'ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "АЛНАЗ"']

In [38]:
result = open('result.txt', 'w')
for company in test_companies:
    result.write(transform(company, clf.predict(extract_features(company, test=True))) + '\n')

ValueError: invalid mode: '\w+'